In [1]:
from keras.applications import inception_v3
from keras import backend as K

K.set_learning_phase(0) # 모델을 훈련하지 않음. 이 명령은 모든 훈련 연산을 비활성화한다

# 합성곱 기반 층만 사용한 인셉션 V3 네트워크를 만든다. 사전 훈련된 ImageNet 가중치와 함께 모델을 로드
model = inception_v3.InceptionV3(weights='imagenet', include_top=False)

87916544/87910968 [==============================] - 31s 0us/step


In [2]:
# 딥드립 설정
layer_contributions = {
    'mixed2' : 0.2,
    'mixed3' : 3.,
    'mixed4' : 2.,
    'mixed5' : 1.5,
}

In [3]:
# 최대화할 손실 정의
layer_dict = dict([(layer.name, layer) for layer in model.layers])
loss = K.variable(0.)
for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output
    
    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    loss += coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling

In [4]:
# 경사 상승법 과정
dream = model.input

grads = K.gradients(loss, dream)[0]

grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values

def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...', i,  '번째 손실 :', loss_value)
        x += step * grad_values
    return x

NameError: name 'preprocess_image' is not defined

In [8]:
import scipy
from keras.preprocessing import image

def resize_img(img, size):
    img = np.copy(img)
    factors = (1, float(size[0]) / img.shape[1], float(size[1]) / img.shape[2], 1)
    return scipy.ndimage.zoom(img, factors, order=1)

def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    image.save_img(fname, pil_img)
    
def preprocess_image(image_path):
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img

def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
        
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [9]:
import numpy as np

step = 0.01
num_octave = 3
octave_scale = 1.4
iterations = 20

max_loss = 10.

base_image_path = './datasets/original_photo_deep_dream.jpg'

img = preprocess_image(base_image_path)

original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i))
                  for dim in original_shape])
    successive_shapes.append(shape)
    
successive_shapes = successive_shapes[::-1]

original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
    print('처리할 이미지 크기', shape)
    img = resize_img(img, shape)
    img = gradient_ascent(img, iterations=iterations, step=step, max_loss=max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    same_size_original = resize_img(original_img, shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img
    img += lost_detail
    shrunk_original_img = resize_img(original_img, shape)
    save_img(img, fname='dream_ac_scale_' + str(shape) + '.png')

save_img(img, fname='./datasets/final_dream.png')

처리할 이미지 크기 (326, 489)
... 0 번째 손실 : 1.8897003
... 1 번째 손실 : 2.3358006
... 2 번째 손실 : 3.004749
... 3 번째 손실 : 3.802278
... 4 번째 손실 : 4.6130652
... 5 번째 손실 : 5.426471
... 6 번째 손실 : 6.155444
... 7 번째 손실 : 6.8558655
... 8 번째 손실 : 7.5736237
... 9 번째 손실 : 8.192286
... 10 번째 손실 : 8.856197
... 11 번째 손실 : 9.454672
처리할 이미지 크기 (457, 685)


/home/gamgoon/development/virtualenv/python3/lib/python3.6/site-packages/scipy/ndimage/interpolation.py:616: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


... 0 번째 손실 : 3.3431816
... 1 번째 손실 : 4.6471386
... 2 번째 손실 : 5.823791
... 3 번째 손실 : 6.824081
... 4 번째 손실 : 7.7534227
... 5 번째 손실 : 8.567659
... 6 번째 손실 : 9.35476
처리할 이미지 크기 (640, 960)
... 0 번째 손실 : 3.3133187
... 1 번째 손실 : 4.5579166
... 2 번째 손실 : 5.6907835
... 3 번째 손실 : 6.7019
... 4 번째 손실 : 7.6789784
... 5 번째 손실 : 8.592467
... 6 번째 손실 : 9.446095
